In [1]:
using PowerSystems
using PowerModels
using PowerModelsInterface
using Ipopt
using Dates
using DataFrames
using Gadfly
using PowerSystemCaseBuilder

In [7]:
sys = build_system(SIIPExampleSystems, "5_bus_matpower_DA")
pm_map = PowerModelsInterface.get_pm_map(sys)

┌ Info: Loaded time series from storage file existing=5_bus_matpower_DA_time_series_storage.h5 new=/tmp/jl_DjI8Q0 compression=CompressionSettings(false, CompressionTypes.DEFLATE = 1, 3, true)
└ @ InfrastructureSystems /home/sam/.julia/packages/InfrastructureSystems/7AXsY/src/hdf5_time_series_storage.jl:98
┌ Warning: Rate 1000.0 MW for bus1-bus5-i_3 is larger than the max expected in the range of (min = 134.0, max = 145.0).
└ @ PowerSystems /home/sam/.julia/packages/PowerSystems/a9ndT/src/utils/IO/branchdata_checks.jl:148
┌ Warning: Rate 200.0 MW for bus1-bus4-i_2 is larger than the max expected in the range of (min = 134.0, max = 145.0).
└ @ PowerSystems /home/sam/.julia/packages/PowerSystems/a9ndT/src/utils/IO/branchdata_checks.jl:148
┌ Warning: Rate 200.0 MW for bus1-bus2-i_1 is larger than the max expected in the range of (min = 134.0, max = 145.0).
└ @ PowerSystems /home/sam/.julia/packages/PowerSystems/a9ndT/src/utils/IO/branchdata_checks.jl:148
┌ Warning: Rate 200.0 MW for bus4-b

Dict{String, Dict} with 7 entries:
  "dcline"  => Dict{Any, Any}()
  "branch"  => Dict{String, ACBranch}("4"=>Line(bus1-bus4-i_2, true, 0.0, 0.0, …
  "gen"     => Dict{String, Generator}("4"=>ThermalStandard(Park City, true, tr…
  "shunt"   => Dict{Any, Any}()
  "storage" => Dict{Any, Any}()
  "load"    => Dict{String, PowerLoad}("1"=>PowerLoad(bus4, true, Bus(4, bus4, …
  "bus"     => Dict{String, Bus}("4"=>Bus(4, bus4, BusTypes.REF = 4, 0.0, 1.0, …

In [8]:
buses = collect(get_components(Bus,sys))

5-element Vector{Bus}:
 Bus(1, bus1, BusTypes.PV = 3, 0.048935017968641414, 1.0, (min = 0.9, max = 1.1), 230.0, Area(1, 0.0, 0.0, 0.0), LoadZone(1, 10.0, 3.2869), Dict{String, Any}())
 Bus(4, bus4, BusTypes.REF = 4, 0.0, 1.0, (min = 0.9, max = 1.1), 230.0, Area(1, 0.0, 0.0, 0.0), LoadZone(1, 10.0, 3.2869), Dict{String, Any}())
 Bus(10, bus5, BusTypes.PV = 3, 0.06266307973312801, 1.0, (min = 0.9, max = 1.1), 230.0, Area(1, 0.0, 0.0, 0.0), LoadZone(1, 10.0, 3.2869), Dict{String, Any}())
 Bus(2, bus2, BusTypes.PQ = 2, -0.012822061349776342, 1.08407, (min = 0.9, max = 1.1), 230.0, Area(1, 0.0, 0.0, 0.0), LoadZone(1, 10.0, 3.2869), Dict{String, Any}())
 Bus(3, bus3, BusTypes.PV = 3, -0.00976895688926266, 1.0, (min = 0.9, max = 1.1), 230.0, Area(1, 0.0, 0.0, 0.0), LoadZone(1, 10.0, 3.2869), Dict{String, Any}())

In [9]:
T = get_forecast_horizon(sys) 
println("Forecast horizon: ",T)

Forecast horizon: 48


## Make the Case 5 Jacobian with PowerModels NR Model

In [10]:
pm_basic_network = make_basic_network(pm_map)
jac_full = calc_basic_jacobian_matrix(pm_basic_network)
jac_full

LoadError: MethodError: no method matching get(::ThermalStandard, ::String, ::Int64)
[0mClosest candidates are:
[0m  get([91m::DataStructures.Trie[39m, ::AbstractString, ::Any) at /home/sam/.julia/packages/DataStructures/nBjdy/src/trie.jl:74
[0m  get([91m::Base.Iterators.Pairs[39m, ::Any, ::Any) at iterators.jl:272
[0m  get([91m::MathOptInterface.Utilities.CleverDicts.CleverDict[39m, ::Any, ::Any) at /home/sam/.julia/packages/MathOptInterface/YDdD3/src/Utilities/CleverDicts.jl:168
[0m  ...

In [11]:

buses = get_components(Bus, sys)
T = get_forecast_horizon(sys)
results = Dict(
    zip(
        get_name.(buses),
        [
            Dict("va" => zeros(T), "vm" => zeros(T), "pg" => zeros(T), "qg" => zeros(T)) for b = 1:length(buses)
        ],
    ),
)
for t = 1:get_forecast_horizon(sys)
    #run the PF
    soln = run_pf(
        sys,
        ACPPowerModel,
        Ipopt.Optimizer,
        start_time = get_forecast_initial_timestamp(sys),
        period = t,
    )
    #extract the results
    for (id, b) in results
        b["va"][t] += soln["solution"]["bus"][id]["va"]
        b["vm"][t] += soln["solution"]["bus"][id]["vm"]
        for (gid, g) in pm_map["gen"]
            if get_name(get_bus(g)) == id
                b["pg"][t] += soln["solution"]["gen"][gid]["pg"]
                b["qg"][t] += soln["solution"]["gen"][gid]["qg"]
            end
        end
    end
end

#convert results into a dataframe
results_dfs = Dict()
for (b, r) in results
    results_dfs[b] = DataFrame(r)
end

results_dfs["1"]


┌ Info: Unit System changed to UnitSystem.SYSTEM_BASE = 0
└ @ PowerSystems /home/sam/.julia/packages/PowerSystems/a9ndT/src/base.jl:339


LoadError: MethodError: no method matching add_pm_cost!(::Dict{String, Any}, ::TwoPartCost)
[0mClosest candidates are:
[0m  add_pm_cost!(::Dict{String, Any}, ::T, [91m::Float64[39m) where T<:TwoPartCost at /home/sam/.julia/packages/PowerModelsInterface/OedyI/src/devices/gen.jl:13
[0m  add_pm_cost!(::Dict{String, Any}, [91m::T[39m, [91m::Float64[39m) where T<:ThreePartCost at /home/sam/.julia/packages/PowerModelsInterface/OedyI/src/devices/gen.jl:2

In [ ]:

# for the OPF approach, we can just run one multi network OPF
mn_soln = run_mn_opf(
    sys,
    ACPPowerModel,
    Ipopt.Optimizer,
    start_time = get_forecast_initial_timestamp(sys),
    time_periods = 1:T,
)

#TODO: extract the data from mn_solution


In [12]:
get_forecast_horizon(sys)

0

In [11]:
results_dfs

Dict{Any, Any} with 5 entries:
  "4"  => 0×4 DataFrame
  "1"  => 0×4 DataFrame
  "10" => 0×4 DataFrame
  "2"  => 0×4 DataFrame
  "3"  => 0×4 DataFrame